<a href="https://colab.research.google.com/github/aleksandrabliznyuchenko/CompLing_2020/blob/main/hw6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2021-02-20 21:21:07--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2021-02-20 21:21:07--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbb8182d2d05d20489649a31c40.dl.dropboxusercontent.com/cd/0/inline/BJW-hgNRLz-dpUOj8UyZCNXo2dGWCuD8YjoHqoltT-1_C4g_fzS_BBSxj2f3pq5ZW5sQ_nTTfgezSWeFi_CWUEto2a_-BwebA0LdEkEO_15Z-w/file# [following]
--2021-02-20 21:21:07--  https://ucbb8182d2d05d20489649a31c40.dl.dropboxusercontent.com/cd/0/inline/BJW-hgNRLz-dpUOj8UyZCNXo2dGWCuD8YjoHqoltT-1_C4g_fzS_BBSxj2f3pq5ZW5sQ_nTTfgezSWeFi_CWUEto2a_-BwebA0LdEkEO_15Z-w/file
Resolving ucbb81

In [2]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv

--2021-02-20 21:21:08--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2021-02-20 21:21:08--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc719dbf8c64843a6d743f7a7257.dl.dropboxusercontent.com/cd/0/inline/BJXMFlo1mujiWDbvDzo_GCeTcc_HHDnLtoek32lixT23hjEFl5fDAMB-rvAf4O1t-cVdMFdklRQpJT8wdu4jhPTG3mjBHLoMjo8txQwv5Wvhvg/file# [following]
--2021-02-20 21:21:09--  https://uc719dbf8c64843a6d743f7a7257.dl.dropboxusercontent.com/cd/0/inline/BJXMFlo1mujiWDbvDzo_GCeTcc_HHDnLtoek32lixT23hjEFl5fDAMB-rvAf4O1t-cVdMFdklRQpJT8wdu4jhPTG3mjBHLoMjo8txQwv5Wvhvg/file
Resolving uc719d

In [3]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, LSTM, Bidirectional, TimeDistributed, Conv1D, concatenate, Flatten
from tensorflow.keras.layers import Embedding, Input
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import Accuracy, Recall, Precision

Обученные эмбеддинги для русского языка

In [4]:
!wget https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz

--2021-02-20 21:21:14--  https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691248108 (2.5G) [application/x-gzip]
Saving to: ‘araneum_none_fasttextcbow_300_5_2018.tgz.1’

araneum_none_fastte 100%[===================>]   2.51G  31.8MB/s    in 81s     

2021-02-20 21:22:36 (31.5 MB/s) - ‘araneum_none_fasttextcbow_300_5_2018.tgz.1’ saved [2691248108/2691248108]



In [5]:
!tar --gunzip --extract --verbose --file=araneum_none_fasttextcbow_300_5_2018.tgz

araneum_none_fasttextcbow_300_5_2018.model
araneum_none_fasttextcbow_300_5_2018.model.vectors_ngrams.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors_vocab.npy


In [6]:
model_file = 'araneum_none_fasttextcbow_300_5_2018.model'
model = KeyedVectors.load(model_file)

##**Подготовка данных**

In [7]:
colnames = ['id', 'date', 'author', 'text', 'type', 'reply', 'retwt', 'fav',  
            'stcount', 'fol', 'friends', 'listcount']

In [8]:
pos = pd.read_csv('positive.csv', sep=';', names=colnames)

In [9]:
neg = pd.read_csv('negative.csv', sep=';', names=colnames)

In [10]:
pos.head()

,id,date,author,text,type,reply,retwt,fav,stcount,fol,friends,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [11]:
neg.head()

,id,date,author,text,type,reply,retwt,fav,stcount,fol,friends,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [12]:
corpus = pd.concat([pos, neg])

In [13]:
corpus.head()

,id,date,author,text,type,reply,retwt,fav,stcount,fol,friends,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


##**Preprocessing**

Уберём имена пользователей и прочие некириллические символы

In [14]:
sentences = []
types = []

def preprocess(text, cur_type):
    if cur_type != 1:
      cur_type = 0
    text = text.lower()
    text = re.sub('[^а-я]', ' ', text)
    types.append([cur_type for i in range(len(text.strip().split()))])
    return text.strip().split()

sentences = [preprocess(row['text'], row['type']) for _, row in corpus.iterrows()]

In [15]:
sentences[0]

['хоть',
 'я',
 'и',
 'школота',
 'но',
 'поверь',
 'у',
 'нас',
 'то',
 'же',
 'самое',
 'общество',
 'профилирующий',
 'предмет',
 'типа']

In [16]:
types[0]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [17]:
len(sentences[0]) == len(types[0])

True

In [18]:
sent_train, sent_test, type_train, type_test = train_test_split(sentences, types, test_size=0.2, random_state=1)

##**Подготовка словарей**

In [19]:
vocab = Counter()
for sentence in sent_train:
    vocab.update(sentence)

In [20]:
filtered_vocab = {word for word in vocab if vocab[word] > 5}

In [21]:
len(filtered_vocab)

23081

In [22]:
word2id = {word:i + 2 for i, word in enumerate(filtered_vocab)}
word2id['pad'] = 0
word2id['unk'] = 1

id2word = {i:word for word, i in word2id.items()}

In [23]:
len(id2word)

23083

In [24]:
weights = np.array([model[word] for word in id2word.values() if word in model])

In [25]:
len(id2word) == len(weights)

True

In [26]:
type2id = {'pad':0, 'unk':1}  
for cur_type in type_train:
    for t in cur_type: 
        if t not in type2id:
            type2id[t] = len(type2id)

id2type = {i:t for t, i in type2id.items()}

In [27]:
id2type

{0: 'pad', 1: 'unk', 2: 1, 3: 0}

In [28]:
chars = set([letter for word in filtered_vocab for letter in word])
n_chars = len(chars)
print(chars)

{'ж', 'р', 'ш', 'з', 'к', 'о', 'е', 'я', 'ъ', 'х', 'т', 'в', 'ы', 'ь', 'с', 'и', 'д', 'л', 'щ', 'ю', 'м', 'ч', 'п', 'ц', 'б', 'г', 'ф', 'н', 'а', 'й', 'э', 'у'}


In [29]:
char2id = {c: i + 2 for i, c in enumerate(chars)}
char2id["pad"] = 0
char2id["unk"] = 1

id2char = {i:char for char, i in char2id.items()}

In [30]:
id2char

{0: 'pad',
 1: 'unk',
 2: 'ж',
 3: 'р',
 4: 'ш',
 5: 'з',
 6: 'к',
 7: 'о',
 8: 'е',
 9: 'я',
 10: 'ъ',
 11: 'х',
 12: 'т',
 13: 'в',
 14: 'ы',
 15: 'ь',
 16: 'с',
 17: 'и',
 18: 'д',
 19: 'л',
 20: 'щ',
 21: 'ю',
 22: 'м',
 23: 'ч',
 24: 'п',
 25: 'ц',
 26: 'б',
 27: 'г',
 28: 'ф',
 29: 'н',
 30: 'а',
 31: 'й',
 32: 'э',
 33: 'у'}

In [31]:
char_max_len = max(len(x) for x in filtered_vocab)
print("Максимальная длина слова: %d" % char_max_len)

Максимальная длина слова: 33


In [32]:
def data2ints(data, smth2id):
  int_data = []
  for seq in data:
      int_seq = []
      for i in seq:
        int_seq.append(smth2id.get(i, 1))
  
      int_data.append(int_seq)
  return int_data

In [33]:
X_train_ids, X_test_ids = data2ints(sent_train, word2id), data2ints(sent_test, word2id)
y_train_ids, y_test_ids = data2ints(type_train, type2id), data2ints(type_test, type2id)

print(X_train_ids[0])
print(y_train_ids[0])

print(X_test_ids[0])
print(y_test_ids[0])

[21721, 1248, 22478, 18401, 19702, 1329, 20061, 21721, 8312, 1040, 786, 14481, 17907, 1647, 1, 2208, 21004, 10250]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[21604, 6816, 1484, 2208, 15269, 1, 1577, 2208, 11747, 1]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [34]:
sent_max_len = max(len(x) for x in X_train_ids)
print("Максимальная длина предложения: %d" % sent_max_len)

Максимальная длина предложения: 34


In [35]:
X_train, X_test = pad_sequences(X_train_ids, maxlen=sent_max_len, padding='post'), pad_sequences(X_test_ids, maxlen=sent_max_len, padding='post')
y_train, y_test = pad_sequences(y_train_ids, maxlen=sent_max_len, padding='post'), pad_sequences(y_test_ids, maxlen=sent_max_len, padding='post')

In [36]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(181467, 34) (181467, 34) (45367, 34) (45367, 34)


In [37]:
def make_X_char(sentences):
  X_char = []
  for sentence in sentences:
      sent_seq = []
      for i in range(sent_max_len):
          word_seq = []
          for j in range(char_max_len):
              try:
                  word_seq.append(char2id[sentence[i][j].lower()])
              except:
                  word_seq.append(char2id["pad"])
          sent_seq.append(word_seq)
      X_char.append(np.array(sent_seq))
  return np.array(X_char)

In [38]:
X_char_train, X_char_test = make_X_char(sent_train), make_X_char(sent_test)

In [39]:
X_char_train.shape

(181467, 34, 33)

Проверим, что мы правильно перевели текст в индексы.

In [40]:
sent_train[5][0]

'хахахахаха'

In [41]:
X_char_train[5][0]

array([11, 30, 11, 30, 11, 30, 11, 30, 11, 30,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [42]:
[id2char[i] for i in X_char_train[5][0]]

['х',
 'а',
 'х',
 'а',
 'х',
 'а',
 'х',
 'а',
 'х',
 'а',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad']

In [43]:
X_train[5]

array([ 8428,  7711,  3650,  7340, 13626,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0], dtype=int32)

In [44]:
[id2word[i] for i in X_train[5]][:50]

['хахахахаха',
 'согласна',
 'будет',
 'полный',
 'жесткач',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad',
 'pad']

In [45]:
y_train, y_test = to_categorical(y_train, num_classes=len(type2id)), to_categorical(y_test, num_classes=len(type2id))

In [46]:
y_train.shape

(181467, 34, 4)

##**biLSTM Model с обучением эмбеддингов внутри модели**

In [ ]:
word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=20, mask_zero=True)(word_in)
lstm = Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.2))(emb_word)
out = TimeDistributed(Dense(len(type2id), activation="softmax"))(lstm)

model = Model(inputs= word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=[Precision(), Recall(), Accuracy()])

In [48]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 34)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 34, 20)            461660    
_________________________________________________________________
bidirectional (Bidirectional (None, 34, 256)           152576    
_________________________________________________________________
time_distributed (TimeDistri (None, 34, 4)             1028      
Total params: 615,264
Trainable params: 615,264
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=1024, epochs=30, verbose=1)

Epoch 1/30
178/178 [==============================] - 51s 256ms/step - loss: 0.2830 - precision: 0.4961 - recall: 0.3388 - accuracy: 0.0000e+00 - val_loss: 0.1871 - val_precision: 0.7064 - val_recall: 0.7008 - val_accuracy: 0.0000e+00
Epoch 2/30
178/178 [==============================] - 44s 248ms/step - loss: 0.1735 - precision: 0.7363 - recall: 0.7318 - accuracy: 0.0000e+00 - val_loss: 0.1745 - val_precision: 0.7325 - val_recall: 0.7297 - val_accuracy: 0.0000e+00
Epoch 3/30
178/178 [==============================] - 44s 248ms/step - loss: 0.1540 - precision: 0.7763 - recall: 0.7737 - accuracy: 0.0000e+00 - val_loss: 0.1746 - val_precision: 0.7363 - val_recall: 0.7345 - val_accuracy: 0.0000e+00
Epoch 4/30
178/178 [==============================] - 44s 247ms/step - loss: 0.1429 - precision: 0.7959 - recall: 0.7941 - accuracy: 0.0000e+00 - val_loss: 0.1780 - val_precision: 0.7313 - val_recall: 0.7300 - val_accuracy: 0.0000e+00
Epoch 5/30
178/178 [==============================] - 44s 24

##**biLSTM Model с предобученными для русского языка эмбеддингами**

In [ ]:
word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=300, trainable=False, weights=[weights])(word_in)
lstm = Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.2))(emb_word)
out = TimeDistributed(Dense(len(type2id), activation="softmax"))(lstm)

model = Model(inputs= word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=[Precision(), Recall(), Accuracy()])

In [51]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 34)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 34, 300)           6924900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 34, 256)           439296    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 34, 4)             1028      
Total params: 7,365,224
Trainable params: 440,324
Non-trainable params: 6,924,900
_________________________________________________________________


In [52]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=1024, epochs=30, verbose=1)

Epoch 1/30
178/178 [==============================] - 40s 208ms/step - loss: 0.5570 - precision_1: 0.8669 - recall_1: 0.5744 - accuracy: 0.0027 - val_loss: 0.2433 - val_precision_1: 0.8641 - val_recall_1: 0.8308 - val_accuracy: 0.0625
Epoch 2/30
178/178 [==============================] - 37s 208ms/step - loss: 0.2371 - precision_1: 0.8684 - recall_1: 0.8450 - accuracy: 0.0261 - val_loss: 0.2242 - val_precision_1: 0.8721 - val_recall_1: 0.8622 - val_accuracy: 0.0182
Epoch 3/30
178/178 [==============================] - 38s 211ms/step - loss: 0.2224 - precision_1: 0.8726 - recall_1: 0.8637 - accuracy: 0.0182 - val_loss: 0.2192 - val_precision_1: 0.8720 - val_recall_1: 0.8670 - val_accuracy: 2.2691e-06
Epoch 4/30
178/178 [==============================] - 37s 209ms/step - loss: 0.2165 - precision_1: 0.8757 - recall_1: 0.8713 - accuracy: 0.0155 - val_loss: 0.2146 - val_precision_1: 0.8770 - val_recall_1: 0.8744 - val_accuracy: 0.0486
Epoch 5/30
178/178 [==============================] - 37

Посмотрим на показатели accuracy у двух моделей - модель с использованием предобученных эмбеддингов эффективнее. Построим усложнённую модель на предобученных эмбеддингах.

##**biLSTM + CharCNN с предобученными для русского языка эмбеддингами**

In [ ]:
# один вход для слов
word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=300, trainable=False, weights=[weights])(word_in)

# другой вход для символов
char_in = Input(shape=(sent_max_len, char_max_len))
emb_char = TimeDistributed(Embedding(input_dim=len(char2id), output_dim=10, input_length=char_max_len))(char_in)
# свертка на символах (CharRNN) применяется к каждому слову отдельно
char_enc = TimeDistributed(Conv1D(filters=12, kernel_size=3))(emb_char)
char_flat = TimeDistributed(Flatten())(char_enc)

# LSTM проходится по всей последовательности, на каждом шаге беря эмбеддинг слова по словарю + символьный эмбеддинг слова 
x = concatenate([emb_word, char_flat])
main_lstm = Bidirectional(LSTM(units=128, return_sequences=True,
                               recurrent_dropout=0.5))(x)
out = TimeDistributed(Dense(len(type2id), activation="softmax"))(main_lstm)

model = Model(inputs=[char_in, word_in], outputs=out)


optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=[Precision(), Recall(), Accuracy()])

In [57]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 34, 33)]     0                                            
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 34, 33, 10)   340         input_6[0][0]                    
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 34, 31, 12)   372         time_distributed_6[0][0]         
____________________________________________________________________________________________

В ходе эскпериментов уменьшила кол-во эпох до 20, т.к. после 20-ой эпохи показатели стали понемногу падать.

In [59]:
model.fit([X_char_train, X_train], y_train, validation_data=([X_char_test, X_test], y_test), batch_size=1024, epochs=20, verbose=1)

Epoch 1/20
178/178 [==============================] - 51s 288ms/step - loss: 0.1706 - precision_3: 0.9133 - recall_3: 0.9133 - accuracy: 0.0330 - val_loss: 0.1826 - val_precision_3: 0.9050 - val_recall_3: 0.9049 - val_accuracy: 0.0465
Epoch 2/20
178/178 [==============================] - 50s 283ms/step - loss: 0.1688 - precision_3: 0.9146 - recall_3: 0.9146 - accuracy: 0.0184 - val_loss: 0.1874 - val_precision_3: 0.9027 - val_recall_3: 0.9027 - val_accuracy: 0.0670
Epoch 3/20
178/178 [==============================] - 51s 286ms/step - loss: 0.1667 - precision_3: 0.9161 - recall_3: 0.9160 - accuracy: 0.0310 - val_loss: 0.1825 - val_precision_3: 0.9060 - val_recall_3: 0.9059 - val_accuracy: 0.0816
Epoch 4/20
178/178 [==============================] - 51s 287ms/step - loss: 0.1653 - precision_3: 0.9172 - recall_3: 0.9171 - accuracy: 0.0231 - val_loss: 0.1836 - val_precision_3: 0.9054 - val_recall_3: 0.9053 - val_accuracy: 0.0470
Epoch 5/20
178/178 [==============================] - 51s 28

###**Попытка сравнения трёх моделей**

Как мы видим, показатели простых моделей biLSTM повышались в процессе обучения, также результаты работы модели с использованием предобученных эмбеддингов были лучше, чем у первой модели.

У усложнённой модели показатели accuracy сначала скорее ухудшаются, но после прохожения 5 эпохи они начинают постепенно расти. В целом же, качество работы третьей модели превосходит предыдущие, так что предварительно мы можем сделать вывод, что увеличение числа слоёв в модели положительно влияет на её эффективность.

